# VeraGPT (Mistral-Core) Test
Run this notebook in Google Colab to load the Mistral-7B-Instruct model and test prompts.

In [ ]:
# Clone repo (skip if already cloned in Colab)
import os
if not os.path.exists("/content/fusion-gpt"):
    !git clone https://github.com/ankush357159/fusion-gpt.git
%cd /content/fusion-gpt/veraGPT

In [ ]:
# Install dependencies (Colab)
!pip -q install -r requirements.txt

In [ ]:
# Optional: set your Hugging Face token (needed for gated models)
import os
os.environ["HF_TOKEN"] = ""  # <- paste your token or leave blank if not required
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
import torch
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer

token = os.environ.get("HF_TOKEN") or None
weights_dir = Path("/content/context/mistral-weights")
model_source = str(weights_dir) if weights_dir.exists() else MODEL_NAME

tokenizer = AutoTokenizer.from_pretrained(model_source, token=token, use_fast=True)

device_map = "auto" if torch.cuda.is_available() else None
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    model_source,
    token=token,
    torch_dtype=dtype,
    device_map=device_map,
)
if device_map is None:
    model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

In [ ]:
# Save model and tokenizer weights locally (optional)
from pathlib import Path

save_dir = Path("/content/context/mistral-weights")
save_dir.mkdir(parents=True, exist_ok=True)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Saved to: {save_dir.resolve()}")

In [ ]:
def generate_response(user_prompt: str, system_prompt: str = "You are a helpful assistant.") -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    if hasattr(tokenizer, "apply_chat_template"):
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        prompt = f"[SYSTEM] {system_prompt}\n[USER] {user_prompt}\n[ASSISTANT]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )
    generated = output_ids[0][inputs["input_ids"].shape[1] :]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()

In [ ]:
prompt = "Explain zero-shot reasoning in simple terms."
print(generate_response(prompt))